In [1]:
!pip install openai

In [5]:
import pandas as pd
import os
import openai
import pandas as pd
import time
from tqdm import tqdm

openai.api_key = ##################### # 본인이 발급받은 key or 환경 변수로 설정

course_df = pd.read_csv('./lecture_review.csv',encoding='utf-8-sig',index_col=0)
course_df

,강의명,교수명,수강 시기,강평
0,경영정보시스템및실습,김종모,22년 1학기 수강자,이제는 손미애 교수님이 하시겟지만...\n김종모교수님 갓.... 그저 빛
1,경영정보시스템및실습,김종모,22년 1학기 수강자,교수님 말이 정말정말 빠르세요 필기가 불가능할 정도로 빠르심 근데 아캠에 강의저장을...
2,경영정보시스템및실습,김종모,22년 1학기 수강자,시험은 몇문제 빼고 괜찮은데 팀을 잘 만나야함. 교수님 굉장히 좋으시고 수업시간때 ...
3,경영정보시스템및실습,김종모,22년 1학기 수강자,교수님 말씀이 조금 빠르시지만 학점 미화.\n조별과제랑 시험 둘다 꽤 빡세서 엉엉 ...
4,경영정보시스템및실습,김종모,22년 1학기 수강자,성적은 잘 주시는 편인데 성적간 차이가 정말 촘촘하다. 중간기말 그저 그렇게 보고 ...
...,...,...,...,...
378,응용통계및연습2,이호우,17년 2학기 수강자,교수님이 강의 잘하시고 팀플없고 비율 거의 채워주고 배우는거 많고 유익함. 등록금이...
379,응용통계및연습2,이호우,17년 2학기 수강자,학부수준에서 배울 수 있는 통계의 끝.\n들으세요
380,응용통계및연습2,이호우,15년 2학기 수강자,시경의 클레멘타인과 같은 수업\n졸업전에 꼭 듣고 졸업하시길
381,응용통계및연습2,이호우,16년 2학기 수강자,다 듣고 눈물을 흘렸습니다. 정말 최고의 과목입니다.


In [6]:
len(course_df)

383

In [7]:
hash_tag_data_col_name = ['해쉬태그']
hash_tag_data = []

# 각 강의의 수강평에 대한 hash tag 생성(분당 3개 까지만 가능... 이건 좀;; -> 최근 데이터 몇 개까지만 고려하도록 하는 것이 좋아보임)
# or 글자수 보면서 강평 몇개를 합치는 방향으로 하는 것도... 일종의 batch process 느낌으로
# or Hash Tag 생성에 대해 다른 자연어 처리 모델 학습...?
############# !!!!!!!!!!! 그냥 한달에 20$ 내면 해결됨...! (분당 60개 까지 생성 가능) ###############################
for data in tqdm(course_df[course_df['교수명']=='김종모']['강평']):    
    temp_message = [{"role" : "user",
                    "content" : f"지금부터 내가 강의평을 입력해줄테니, 해당 강의를 수강하는 학생들에게 유용한 정보들을 해쉬 태그 형태로 아주 짧게 제시해라. '#과제 없음 #시험 매우 쉬움 #학점 천사'와 같이 아주 짧지만 수강신청에 필요한 해당 강의의 정보를 모두 포함해서 말해라. 따라서, 너는 내게 '#해쉬태그정보' 형태인 정보만 제공해주면 된다. \n\n강의평 : {data}"}]    
    temp_inference = openai.ChatCompletion.create(model="gpt-3.5-turbo",messages=temp_message)
    hash_tag_data.append(temp_inference.choices[0].message['content'].strip())
    time.sleep(25)
    
# 강의평, 요약한 해쉬태그
hash_tag_df = pd.DataFrame(hash_tag_data, columns=hash_tag_data_col_name)
total_df = pd.concat([course_df,hash_tag_df],axis=1)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [04:28<00:00, 26.85s/it]


In [9]:
total_df

,강의명,교수명,수강 시기,강평,해쉬태그
0,경영정보시스템및실습,김종모,22년 1학기 수강자,이제는 손미애 교수님이 하시겟지만...\n김종모교수님 갓.... 그저 빛,#김종모교수님 #갓 #학점 잘 나옴
1,경영정보시스템및실습,김종모,22년 1학기 수강자,교수님 말이 정말정말 빠르세요 필기가 불가능할 정도로 빠르심 근데 아캠에 강의저장을...,#교수님말빠름 #강의저장유무불확실 #교수님ppt많음 #필기중요 #개인과제1번 #팀플...
2,경영정보시스템및실습,김종모,22년 1학기 수강자,시험은 몇문제 빼고 괜찮은데 팀을 잘 만나야함. 교수님 굉장히 좋으시고 수업시간때 ...,#팀프로젝트 필수 #좋은 교수님 #집중력 필요 #정리 자료 필수 #성적 잘 받기 쉬움
3,경영정보시스템및실습,김종모,22년 1학기 수강자,교수님 말씀이 조금 빠르시지만 학점 미화.\n조별과제랑 시험 둘다 꽤 빡세서 엉엉 ...,#학점 미화 #조별과제 빡세다 #시험 어렵다 #손미애교수님 추천
4,경영정보시스템및실습,김종모,22년 1학기 수강자,성적은 잘 주시는 편인데 성적간 차이가 정말 촘촘하다. 중간기말 그저 그렇게 보고 ...,"#성적 주는데 촘촘 #중간, 기말 그저그럼 #팀플 잘 걸려서 학점 잘 챙길 수 있음"
...,...,...,...,...,...
378,응용통계및연습2,이호우,17년 2학기 수강자,교수님이 강의 잘하시고 팀플없고 비율 거의 채워주고 배우는거 많고 유익함. 등록금이...,NaN
379,응용통계및연습2,이호우,17년 2학기 수강자,학부수준에서 배울 수 있는 통계의 끝.\n들으세요,NaN
380,응용통계및연습2,이호우,15년 2학기 수강자,시경의 클레멘타인과 같은 수업\n졸업전에 꼭 듣고 졸업하시길,NaN
381,응용통계및연습2,이호우,16년 2학기 수강자,다 듣고 눈물을 흘렸습니다. 정말 최고의 과목입니다.,NaN


In [8]:
total_df.to_csv('./lecture_review_with_hashtag_df.csv',encoding='utf-8-sig')

In [11]:
# 강의평별 해쉬태그 -> 요약 강의 전체 해쉬 태그
temp_data = ""
for data in total_df[total_df['교수명']=='김종모']['해쉬태그']:
    temp_data = temp_data + " " + data

message = [{"role" : "user",
           "content" : f"아래는 수강평을 해쉬태그 형태로 핵심 정보들만 짧게 추출한 결과야. 아래의 해쉬태그들을 고려했을 때, 공통적으로 제시할 수 있는 핵심 정보를 짧은 해쉬태그 형태로 반드시 5개를 넘지 않도록 제시해줘. 따라서, 너는 최대 5개까지 '#해쉬태그정보' 형태로 제시하면 되는거야. 절대 부가적인 설명을 덧붙이지마. \n \n {temp_data}"}]
inference = openai.ChatCompletion.create(model="gpt-3.5-turbo",messages=message)
result = inference.choices[0].message['content'].strip()

In [12]:
# 성능 별로네... 강평별 해쉬태그 보면서 요약하는 게 좋을 것 같기도?
# 이건 chatGPT Plus로 쫙 뽑은 다음에 input message 몇번 조절해보자!
result

'#갓종모 #성적잘줌 #팀프로젝트 #손미애교수님추천 #에이쁠안락합니다'